# Word2Vec

## 분산 표현(Distributed Representation)
- 분포 가설(distributional hypothesis)이라는 가정 하에 만들어진 표현 방법 
- '비슷한 위치에서 등장하는 단어들은 비슷한 의미를 가진다'라는 가정
- 원-핫 벡터처럼 벡터의 차원이 단어 집합(vocabulary)의 크기일 필요가 없으므로, 벡터의 차원이 상대적으로 저차원으로 줄어듬
- 단어의 의미를 여러 차원에다가 분산하여 표현
- 단어 간 유사도를 계산할 수 있음
- NNLM, RNNLM, Word2Vec

## CBOW(Continuous Bag of Words)
- CBOW는 주변에 있는 단어들을 가지고, 중간에 있는 단어들을 예측하는 방법
- Ex : "The fat cat sat on the mat" => 주변 단어(context word) {"The", "fat", "cat", "on", "the", "mat"}으로부터 중심 단어(center word) sat을 예측
- window : 중심 단어를 예측하기 위해서 앞, 뒤로 몇 개의 단어를 볼지를 결정하는 범위
- sliding window : 윈도우를 계속 움직여서 주변 단어와 중심 단어 선택을 바꿔가며 학습을 위한 데이터 셋을 만듬
- 입력 : 앞, 뒤로 사용자가 정한 윈도우 크기 범위 안에 있는 주변 단어들의 원-핫 벡터
- 출력 : 예측하고자 하는 중간 단어의 원-핫 벡터
- 딥러닝이 아니라 1개의 은닉층만을 사용한 Shallow Neural Network + 활성화 함수 존재하지 않음
- loss : cross entropy

## Skip-gram
- Skip-Gram은 중간에 있는 단어로 주변 단어들을 예측하는 방법

## 네거티브 샘플링(Negative Sampling)
Skip-gram으로 학습할때, 출력층에 있는 소프트맥스 함수는 단어 집합 크기의 벡터 내의 모든 값을 0과 1사이의 값이면서 모두 더하면 1이 되도록 바꾸는 작업을 수행 
- 단어 집합의 크기가 수백만에 달한다면 이 작업은 굉장히 무거운 작업
- 역전파를 수행하므로 주변 단어와 상관 없는 모든 단어까지의 워드 임베딩 조정 작업을 수행 => 관계가 없는 수많은 단어의 임베딩을 조정할 필요가 없음

위 문제를 해결하기 위해 전체 단어 집합보다 훨씬 작은 단어 집합을 만들어놓고 마지막 단계를 이진 분류 문제로 바꿈 
- 주변단어로 이루어진 집합 A와 랜덤으로 선택된 주변단어가 아닌 집합 B
- 주변 단어들을 긍정(positive)으로 두고 랜덤으로 샘플링 된 단어들을 부정(negative)으로 둔 다음에 이진 분류 문제를 수행

## 영어 Word2Vec 만들기

### 훈련 데이터 획득

In [1]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x1ad63080130>)

### 전처리
- 필요한 데이터는 영어문장으로만 구성된 내용을 담고 있는 < content >와 < /content > 사이의 내용
- xml 문법들은 제거
- (Laughter)나 (Applause)와 같은 배경음을 나태나는 단어 제거

In [11]:
from lxml import etree
import re
import zipfile
from nltk.tokenize import word_tokenize, sent_tokenize

In [6]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()')) # xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

In [10]:
content_text = re.sub(r'\([^)]*\)', '', parse_text) # 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.

In [12]:
sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

In [16]:
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

In [17]:
result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.

print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [18]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


### Word2Vec 훈련시키기

In [20]:
from gensim.models import Word2Vec, KeyedVectors

In [21]:
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram.

In [22]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8305555582046509), ('guy', 0.8077905178070068), ('lady', 0.7679110765457153), ('boy', 0.7539355754852295), ('gentleman', 0.7429524660110474), ('girl', 0.7246814370155334), ('soldier', 0.6980609893798828), ('kid', 0.6841074228286743), ('surgeon', 0.6815424561500549), ('poet', 0.6681637763977051)]


man과 유사한 단어로 woman, guy, boy, lady, girl, gentleman, soldier, kid 등을 출력

### Word2Vec 모델 저장하고 로드

In [23]:
model.wv.save_word2vec_format('./eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [24]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8305555582046509), ('guy', 0.8077905178070068), ('lady', 0.7679110765457153), ('boy', 0.7539355754852295), ('gentleman', 0.7429524660110474), ('girl', 0.7246814370155334), ('soldier', 0.6980609893798828), ('kid', 0.6841074228286743), ('surgeon', 0.6815424561500549), ('poet', 0.6681637763977051)]


## 임베딩 벡터의 시각화(Embedding Visualization)

### 워드 임베딩 모델로부터 2개의 tsv 파일 생성
python -m gensim.scripts.word2vec2tensor --input 모델이름 --output 모델이름   
python -m gensim.scripts.word2vec2tensor --input eng_w2v --output eng_w2v을 cmd에서 실행

### 임베딩 프로젝터를 사용하여 시각화하기
구글의 [Embedding Projector](https://projector.tensorflow.org/)을 사용하여 워드 임베딩 모델 시각화